In [0]:
# Imports
import requests
from bs4 import BeautifulSoup
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output
from warnings import warn
import pandas as pd

# Use your own User-Agent
headers = {'User-Agent':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36 RuxitSynthetic/1.0 v316542848 t18859'}

In [0]:
# Get the tv urls for the overview details page
# Lists to store the scraped data in
tv_urls = []

# Get series urls - the overview of the series 
# Preparing the monitoring of the loop
start_time = time()
req = 0

# The loop
categories = ['new-series', 'returning-series', 'special-event']
# For every category
for category in categories:
  if category=='returning-series':
    pages = range(7)
  else:
    pages = [0]

  # For every page
  for page in pages:
    if page==0:
      response = requests.get('https://www.metacritic.com/browse/tv/release-date/'+category+'/date', headers=headers)
    else:
      response = requests.get('https://www.metacritic.com/browse/tv/release-date/'+category+'/date?page='+str(page), headers=headers)

    # Pause the loop
    sleep(randint(8,15))

    # Monitor the requests
    req += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(req, req/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
      warn('Request: {}; Status code: {}'.format(req, response.status_code))
    
    # Break the loop if the number of requests is greater than expected
    if req > 10:
      warn('Number of requests was greater than expected.')
    
    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')

    # Select all the tvshow containters from a single page
    first_show = page_html.find('li', class_='product season_product first_product')
    show_containers = page_html.find_all('li', class_='product season_product')
    last_show = page_html.find('li', class_='product season_product last_product')

    # Extract the page of tvshow from individual tvshow container
    # First container
    # TV Show page url
    show_url = first_show.a['href'].split('/season-', 1)[0]
    if show_url not in tv_urls:
      tv_urls.append(show_url)

    # Middle containers
    for container in show_containers:
      show_url = container.a['href'].split('/season-', 1)[0]
      if show_url not in tv_urls:
        tv_urls.append(show_url)

    # Last container
    # TV Show page url
    show_url = last_show.a['href'].split('/season-', 1)[0]
    if show_url not in tv_urls:
      tv_urls.append(show_url)

In [0]:
# Save tv_urls into csv, so that we don't need to make the future requests
tv_urls_df = pd.DataFrame({'tv_url': tv_urls})
tv_urls_df.to_csv('tv_urls.csv', index=False)

In [26]:
# Import the `tv_urls.csv` file
from google.colab import files
uploaded = files.upload()

Saving tv_urls.csv to tv_urls (3).csv


In [28]:
import io

# Read into a dataframe
tv_urls = pd.read_csv(io.StringIO(uploaded['tv_urls.csv'].decode("utf-8")))

# Save tv_url in a list
tv_urls_list = tv_urls['tv_url'].tolist()
tv_urls_list[:5]

['/tv/love-fraud',
 '/tv/in-my-skin',
 '/tv/quiz',
 '/tv/dcs-stargirl',
 '/tv/snowpiercer']

In [0]:
# Loop through all tv_urls and get title, date, user names, user ratings, and user reviews
# This takes the information for the overview of the tv series

# Lists to store the scraped data
titles = []
dates = []
user_names = []
ratings = []
reviews = []
review_dates = []

# Preparing the monitoring of the loop
start_time = time()
req = 0

# The loop
for url in tv_urls_list:
  response = requests.get('https://www.metacritic.com'+url+'/user-reviews', headers=headers)

  # Pause the loop
  sleep(randint(8,15))

  # Monitor the requests
  req += 1
  elapsed_time = time() - start_time
  print('Request:{}; Frequency: {} requests/s'.format(req, req/elapsed_time))
  clear_output(wait = True)

  # Throw a warning for non-200 status codes
  if response.status_code != 200:
    warn('Request: {}; Status code: {}'.format(req, response.status_code))
  
  # Parse the content of the request with BeautifulSoup
  page_html = BeautifulSoup(response.text, 'html.parser')

  # The title
  title = page_html.h1.text

  # The release date
  release_date = page_html.find('span', class_='release_date').find('span', class_=None).text

  # The reviews containers
  review_containers = page_html.find_all('div', class_='review pad_top1')

  # Get the user name, rating, review, and review date
  for container in review_containers:
    # Add the title for each container
    titles.append(title)

    # Add the release date for each container
    dates.append(release_date)

    # The user name
    user_name = container.a.text
    user_names.append(user_name)

    # The rating
    rating = container.find('div', class_='left fl').text.replace('\n','')
    ratings.append(rating)

    # The review
    review = container.find('div', class_='review_body').text.replace('\n', '')
    reviews.append(review)

    # The review date
    review_date = container.find('span', class_='date').text
    review_dates.append(review_date)

  # IF NEXT PAGE OF REVIEWS, GO TO THE NEXT PAGE
  # TODO